In [9]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [10]:
from crewai import Agent, Task, Crew

import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

### Tools

In [11]:
import yfinance as yf
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchResults
from crewai_tools import ScrapeWebsiteTool

# YAHOO FINANCE TOOL 
def fetch_stock_price(ticker):
    stock = yf.download(ticker, period="1y")
    return stock


yahoo_finance_tool = Tool(
    name = "Yahoo Finance Tool",
    description = "Fetches stocks prices for {ticker} from the last year about a specific company from Yahoo Finance API",
    func= lambda ticker: fetch_stock_price(ticker)
)

search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)
YF_tool = yahoo_finance_tool
scrape_tool = ScrapeWebsiteTool()

### Agents

In [12]:

## Stock Price Analyst
stock_price_analyst = Agent(
    role='Stock Price Analyst',
    goal="Analyze the stock :{ticker}",
    backstory=(
        "You are highly experienced in analyzing the price of a specific stock, utilizing advanced techniques "
        "and deep knowledge of market trends. With a keen understanding of both fundamental and technical analysis, "
        "you assess factors that influence stock price movements. You leverage historical data, market conditions, and financial reports "
        "to identify patterns and predict future price trajectories. Your expertise allows you to evaluate potential risks and opportunities, " 
        "providing accurate and actionable predictions. Investors rely on your insights to make informed decisions and optimize their investment strategies."
    ),
    tools=[YF_tool],
    verbose=True,
    allow_delegation=False,
)


## Stock News Analyst
stock_news_analyst = Agent(
    role='Stock News Analyst',
    goal="Analyze the news related to the stock :{ticker}",
    backstory=(
        "You are highly experienced in analyzing market trends and news, with over 10 years of tracking assets and market behavior. " 
        "As a master-level analyst in traditional markets, you possess a deep understanding of human psychology and its influence on market " 
        "movements. You carefully evaluate news, titles, and information, approaching them with a healthy dose of skepticism. Always considering " 
        "the source of the news articles, you ensure that your analysis is grounded in credible and reliable information. Your expertise enables you " 
        "to discern valuable insights from noise, providing investors with sound, data-driven advice."
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    allow_delegation=False,
)


## Stock Writer
stock_analyst_writer = Agent(
    role='Stock Analyst Writer',
    goal="Write a comprehensive report on the stock :{ticker}",
    backstory=(
        "You are widely regarded as the best stock analyst in the market, known for your ability to understand and explain complex financial " 
        "concepts. Your skill in crafting compelling stories and narratives makes intricate stock analysis accessible and engaging for a wide " 
        "audience. You seamlessly combine multiple theories, such as cycle theory and fundamental analysis, to provide a holistic view of the " 
        "market. Your approach allows you to hold and evaluate multiple perspectives, ensuring that your analysis is thorough and balanced. " 
        "Investors trust your insights, knowing that your nuanced approach leads to informed and strategic decisions."
    ),
    #tools=[search_tool],
    verbose=True,
    allow_delegation=True,
)



## Tasks

In [17]:
get_stock_price = Task (
    description = "Analyze the stock {ticker} price history to determine if the trend is up, down, or sideways based on recent market movements.",
    expected_output = "Examine the stock's recent price movements to identify its prevailing trend. Assess whether the stock is showing consistent upward momentum, a downward decline, or fluctuating within a range. " 
                        "Based on this analysis, specify if the trend is up, down, or sideways.",
    agent = stock_price_analyst,
)


get_stock_news = Task (
    description = "Collect the latest news articles related to the stock {ticker} and provide a summary of the most recent news articles.",
    expected_output = "Retrieve the most recent news articles related to the stock's ticker symbol. Compile a summary of the most recent news articles, focusing on key information and highlights. "
                        "Ensure that the summary is concise and captures the essence of the news.",
    agent = stock_news_analyst,
)

write_stock_analysis = Task (
    description = "Write a comprehensive analysis of the stock {ticker} based on the provided stock price history and news summary.",
    expected_output = "Craft a detailed analysis of the stock's current price with date, performance and market conditions. Incorporate the stock price history and news summary to provide a " 
                        "comprehensive overview of the stock's current status, potential risks, and opportunities. "
                        "Ensure that the analysis is well-structured, engaging, bullet points and captures the essence of the stock's performance and market dynamics.",
    agent = stock_analyst_writer,
    context = [get_stock_price, get_stock_news]
)

## Crew

In [18]:
from crewai import Process


crew = Crew(
    agents = [stock_price_analyst, stock_news_analyst, stock_analyst_writer],
    tasks = [get_stock_price, get_stock_news, write_stock_analysis],
    #process=Process.hierarchical,
    verbose=2,
    memory=True,
)

2024-12-30 17:35:00,142 - 11416 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [19]:
inputs = {
    "ticker": "MSFT"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Stock Price Analyst
 [INFO]: == Starting Task: Analyze the stock MSFT price history to determine if the trend is up, down, or sideways based on recent market movements.


> Entering new CrewAgentExecutor chain...
I need to analyze the stock price history of MSFT to determine the prevailing trend based on recent market movements.

Action: Yahoo Finance Tool
Action Input: {"ticker": "MSFT"}

[*********************100%***********************]  1 of 1 completed


 

Price            Close        High         Low        Open    Volume
Ticker            MSFT        MSFT        MSFT        MSFT      MSFT
Date                                                                
2023-12-28  373.240112  374.413691  372.126205  373.329619  14327000
2023-12-29  373.995972  375.109879  371.449889  373.956181  18723000
2024-01-02  368.854095  373.856753  364.776375  371.827833  25258600
2024-01-03  368.585571  371.231116  366.506935  367.004218  23083500
2024-01-04  365.940002  371.071958  365.174199  368.655174  20901500
...                ...         ...         ...         ...       ...
2024-12-20  436.600006  443.739990  428.630005  433.109985  64263700
2024-12-23  435.250000  437.649994  432.829987  436.739990  19152500
2024-12-24  439.329987  439.600006  434.190002  434.649994   7164500
2024-12-26  438.109985  440.940002  436.630005  439.079987   8194200
2024-12-27  430.529999  435.220001  426.350006  434.600006  18095100

[252 rows x 5 columns]

Though

2024-12-30 17:35:16,060 - 11416 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--48219f166042b5754512334ac0be1fc50564bdcaed4127ab7cc2a67591c3cac9


I need to gather detailed financial data, competitor analysis, potential risks, and challenges faced by MSFT to provide a comprehensive analysis for my coworker.

Action: Yahoo Finance Tool
Action Input: {"ticker": "MSFT"}

> Entering new CrewAgentExecutor chain...
I need to gather specific financial data, competitor analysis, potential risks, and challenges faced by MSFT for a comprehensive analysis.

Action: Yahoo Finance Tool
Action Input: {"ticker": "MSFT"}

> Entering new CrewAgentExecutor chain...
I need to gather comprehensive financial data, competitor analysis, and potential risks and challenges faced by MSFT to provide a thorough analysis for my coworker.

Action: Yahoo Finance Tool
Action Input: {"ticker": "MSFT"}

> Entering new CrewAgentExecutor chain...
I need to gather specific financial data, competitor analysis, potential risks, and challenges faced by MSFT to provide a comprehensive analysis for my coworker.

Action: Yahoo Finance Tool
Action Input: {"ticker": "MSFT"}

In [16]:
from IPython.display import Markdown
Markdown(result)

Comprehensive Analysis of Microsoft Corporation (MSFT):

Introduction:
Microsoft Corporation (MSFT) is a leading technology company known for its software products, cloud services, and hardware devices. As one of the most valuable companies globally, Microsoft's stock performance is closely watched by investors and analysts alike.

Stock Price Performance:
- Over the past 5 years, MSFT has shown impressive stock price growth, outperforming industry benchmarks.
- The stock has experienced steady growth, with occasional fluctuations due to market conditions and company-specific news.
- MSFT's stock price movement reflects the company's strong financial performance and strategic initiatives.

Financial Analysis:
- MSFT has shown consistent revenue growth, driven by its core software products like Windows and Office, as well as its cloud computing services under Azure.
- Profit margins have remained healthy, showcasing the company's ability to generate significant returns for shareholders.
- Financial ratios indicate a solid financial position, with strong cash reserves and manageable debt levels.

Recent News and Developments:
1. Microsoft Stock Today: MSFT Stock Is Still A Buy In December; Here's Why (Yahoo)
   - Provides insights into the current market sentiment towards MSFT and reasons for its continued growth potential.

2. Microsoft Stock vs. Meta Platforms Stock: Billionaires Are Buying One and Selling the Other (The Motley Fool on MSN.com)
   - Offers a comparison between MSFT and Meta Platforms stock, highlighting investor trends and preferences.

3. Microsoft Stock: Poised To Break Out (Technical Analysis, Upgrade) (Seeking Alpha)
   - Discusses technical analysis and potential upgrades that could impact MSFT's stock performance.

4. Microsoft (MSFT) Price Prediction and Forecast 2025-2030 (24/7 Wall St)
   - Provides a long-term outlook on MSFT's stock price, offering insights for investors planning for the future.

5. Why Is Microsoft Corp (MSFT) Stock Surpassing 2.73% from Its 50-Day SMA? (bovnews)
   - Analyzes stock movements and technical indicators that could influence short-term trading decisions.

6. Analyst reworks Microsoft stock price target in software review (TheStreet on MSN.com)
   - Highlights changes in analyst recommendations and price targets for MSFT, impacting investor sentiment.

7. Microsoft: Backdated Azure Growth And Pricing Discounts Make The Stock Less Attractive (Seeking Alpha)
   - Discusses potential challenges facing MSFT, including concerns about Azure growth and pricing strategies.

8. Microsoft (MSFT): New Buy Recommendation for This Technology Giant (Business Insider)
   - Provides a positive outlook on MSFT, with a new buy recommendation for investors looking to enter the market.

9. Microsoft Corporation (MSFT): Staying Resilient as 2024 Wraps Up (InvestingCube)
   - Reflects on MSFT's performance throughout the year and its ability to navigate market uncertainties effectively.

Conclusion:
Microsoft Corporation (MSFT) continues to be a strong performer in the technology sector, with solid financials, consistent growth, and a positive market outlook. While facing challenges like any other company, MSFT's resilience and strategic positioning make it an attractive investment option for both short-term traders and long-term investors. As always, conducting thorough research and staying informed about market trends will be key for maximizing returns on investments in MSFT.